In [1]:
import pandas as pd 

In [2]:
people = pd.read_csv("EHRShot/sampled_person.csv")
deaths = pd.read_csv("EHRShot/sampled_death.csv")
conditions = pd.read_csv("EHRShot/sampled_condition_occurrence.csv")
concepts = pd.read_csv("EHRShot/concept.csv")

C:\Users\patri\AppData\Local\Temp\ipykernel_21864\2303764578.py:3: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  conditions = pd.read_csv("EHRShot/sampled_condition_occurrence.csv")
C:\Users\patri\AppData\Local\Temp\ipykernel_21864\2303764578.py:4: DtypeWarning: Columns (0,9) have mixed types. Specify dtype option on import or set low_memory=False.
  concepts = pd.read_csv("EHRShot/concept.csv")


In [10]:
# find conditions where the person has an associated record in deaths
mask = conditions['person_id'].isin(set(deaths['person_id']))
conditions_with_deaths = conditions.loc[mask].copy()
conditions_with_deaths

print(f'Conditions total: {len(set(conditions['condition_concept_id']))}, Conditions with deaths: {len(set(conditions_with_deaths['condition_concept_id']))}')
print(f'People total: {len(people)}, People suffering death: {len(set(conditions_with_deaths["person_id"]))}')

Conditions total: 6376, Conditions with deaths: 3120
People total: 2000, People suffering death: 207


In [ ]:
# find people who died in their 20s
# create date of birth column in deaths

# left join people and their birth date to deaths
people_deaths = pd.merge(deaths, people.filter(['person_id', 'year_of_birth']), on='person_id', how='left')
# calculate age at death column
people_deaths['age_at_death'] = people_deaths['death_DATE'].str.slice(0,4).astype(int) - people_deaths['year_of_birth']
people_deaths[people_deaths['age_at_death'].between(20,29)]

,person_id,death_DATE,death_DATETIME,death_type_concept_id,cause_concept_id,cause_source_value,cause_source_concept_id,trace_id,unit_id,load_table_id,_death_date_external,year_of_birth,age_at_death
76,115968882,2014-07-22,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1991,23
82,115973201,2015-02-21,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1988,27
85,115967423,2015-05-12,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1990,25
110,115968171,2017-03-12,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1988,29
114,115972356,2017-08-17,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1993,24
149,115967241,2020-03-20,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1993,27
201,115972311,2022-10-02,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1995,27
